In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# From 3-initial_analysis.py -> the mean of reddit submissions dataset
DATA_SET_MEAN = 37.84489897479949

# Analysis of Data
Conducting analysis of data of the two subsets of reddit submissions data.  
1. Above average submissions (score greater than DATA_SET_MEAN)
2. Below average submissions (subsampled submissions with score less than DATA_SET_MEAN)

In [6]:
above_avg = pd.read_json('./data-subsets/submissions-above-avg.json.gz', lines=True, orient='records', compression='gzip')
below_avg = pd.read_json('./data-subsets/submissions-below-avg.json.gz', lines=True, orient='records', compression='gzip')

## Post Title Analysis

Does the length of the post title have an affect on the submission score?  
Between above average and below average submissions, is there a difference in their title lengths?

## User Post Frequency Analysis

Does the frequency in which a user posts in certain subreddit affect their submission performance?  
Are above average posts from users that frequently post more? Are below average posts from users that post less?

## Post Time Analysis

Does the time in which a post is created affect the performance at all?  
How do submissions scores that are posted during peak reddit usage times differ from posts made outside these times?  
Does the number of posts made in a time period affect the average score of submissions?